# Guided Project: Predicting Board Game Reviews
### Apply clustering and regression techniques to predict board game review scores.

##### Contents:
- KMeans
- set()
- list.remove()
- df.apply(np.mean, axis=1)
- df.corr()
- sklearn.linear_model.LinearRegression
- Other
    - SVR
    - RandomForestRegressor

## 1: Board Game Reviews
Board games have been making a comeback lately, and deeper, more strategic boardgames, like [Settlers of Catan](https://en.wikipedia.org/wiki/Catan) have become hugely popular. A popular site where these types of board games are discussed and reviewed is [BoardGameGeek](http://www.boardgamegeek.com/).

In this project, you'll be working with a data set that contains `80000` board games and their associated review scores. The data was scraped from BoardGameGeek and compiled into CSV format by [Sean Beck](https://github.com/ThaWeatherman). The data set is stored in `board_games.csv`, and can be downloaded [here](https://github.com/ThaWeatherman/scrapers/blob/master/boardgamegeek/games.csv). If you need help at any point, you can consult our solution notebook [here](https://github.com/dataquestio/solutions/blob/master/Mission211Solution.ipynb).

Here's a preview of the first 5 rows and columns:

| id     | type      | name                                      | yearpublished | minplayers |
|--------|-----------|-------------------------------------------|---------------|------------|
| 12333  | boardgame | Twilight Struggle                         | 2005          | 2          |
| 120677 | boardgame | Terra Mystica                             | 2012          | 2          |
| 102794 | boardgame | Caverna: The Cave Farmers                 | 2013          | 1          |
| 25613  | boardgame | Through the Ages: A Story of Civilization | 2006          | 2          |
| 3076   | boardgame | Puerto Rico                               | 2002          | 2          |

Each row represents a single board game, and has descriptive statistics about the board game, as well as review information. Here are some of the interesting columns:

- `name`: name of the board game.
- `playingtime`: the playing time (given by the manufacturer).
- `minplaytime`: the minimum playing time (given by the manufacturer).
- `maxplaytime`: the maximum playing time (given by the manufacturer).
- `minage`: the minimum recommended age to play.
- `users_rated`: the number of users who rated the game.
- `average_rating`: the average rating given to the game by users. (0-10)
- `total_weights`: Number of weights given by users. Read more about what BoardGameGeek considers weights [here](http://boardgamegeek.com/wiki/page/Weight).
- `average_weight`: the average of all the subjective weights (0-5).

One interesting machine learning task might be to predict `average_rating` using the other columns. The data set contains quite a few missing values, and rows where there are no reviews. You'll need to remove these as you explore the data to make prediction easier.

#### Instructions:

- `Read board_games.csv` into a Dataframe called `board_games` using the pandas library.
- Display the first few rows of `board_games` and get familiar with the data set.
- Use the [DataFrame.dropna()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) method to remove rows containing missing values.
- Remove any rows that have no reviews.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
board_games = pd.read_csv('data/board_games.csv')
board_games.head(3)

In [ ]:
board_games.describe()

In [ ]:
board_games.dropna(inplace=True)
board_games = board_games[board_games['users_rated'] > 0]

## 2: Picking An Error Metric

You want to predict the average_rating column using the other columns, but you'll need to do some data exploration before you're ready to do so. The exploration will help you understand the distribution of average_rating better, as well as select an error metric that you'll use to evaluate the performance of your machine learning model.

#### Instructions:
- Become familiar with the distribution of average ratings by generating plots and calculating summary statistics. For each of the following, write down your observations in a Markdown cell.
    - Generate a histogram of the average ratings.
    - Generate a box and whisker plot of the average ratings.
    - Calculate the standard deviation.
    - Calculating the mean.
- Think about what error metric might make sense for this data, and write a markdown cell with your thoughts.

In [ ]:
plt.hist(board_games.average_rating)
plt.show()

In [ ]:
plt.boxplot(board_games.average_rating)
plt.show()

In [ ]:
avg_rating_std = board_games.average_rating.std()
avg_rating_mean = board_games.average_rating.mean()

print(avg_rating_std, avg_rating_mean)

The `average_rating` data is continuous, so MSE could be a good metric since it penalizes larger errors.

## 3: Plotting Clusters

Now that you have a handle on the `average_rating` column, and have picked an error metric, you're ready for the next step. If you haven't picked an error metric, you should look at Mean Squared Error. As the data is continuous, and you want to penalize larger errors more, Mean Squared Error is a good error metric choice.

You can look at the data for patterns that may help you develop a machine learning model. One way to look for patterns is to use a clustering algorithm to create clusters, then plot them out.

You can first use the [sklearn.cluster.KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) class to fit a k-means clustering model. This class only works with numeric columns, so you have to extract the numeric columns of `board_games` before passing them into the [KMeans.fit()](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans.fit) method.

To visualize how board games are clustered, we can calculate the row means and row standard deviations and then generate a scatter plot that compares the means against the standard deviations. We encourage you to think of other ways to visualize the rows as clusters as well.

#### Instructions:

- Use the KMeans class to create clusters.
    - Initialize the KMeans class with 5 clusters.
    - Extract the numeric columns of board_games, and assign to the variable numeric_columns.
        - Leave out name, type, and id.
    - Fit the KMeans class to numeric_columns using the fit method.
    - Extract the labels_ attribute of the KMeans class, and assign to the variable labels.
- Plot out the cluster assignments.
    - Use the apply method on numeric_columns with the keyword argument axis set to 1 to find the mean of each row. Assign the result to game_mean.
    - Use the apply method on numeric_columns with the keyword argument axis set to 1 to find the standard deviation of each row. Assign the result to game_std.
    - Create a plot using the matplotlib scatter function, with the c keyword argument set to labels, the keyword argument x set to game_mean, and the keyword argument y set to game_std.
- What do the results tell you? Write up the results in a markdown cell.

In [ ]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(5)

In [ ]:
numeric_columns = list(board_games._get_numeric_data().columns)
numeric_columns

In [ ]:
set(board_games.columns) - set(numeric_columns)

In [ ]:
numeric_columns.remove('id')
board_games[numeric_columns]

In [ ]:
kmeans_model.fit(board_games[numeric_columns])
labels = kmeans_model.labels_

In [ ]:
game_mean = board_games[numeric_columns].apply(np.mean,axis=1)
game_std = board_games[numeric_columns].apply(np.std, axis=1)

In [ ]:
plt.scatter(game_mean, game_std, c=labels)

## 4: Finding Correlations

Now that you're done some data exploration, you can figure out which columns correlate well with `average_rating`. This will enable you to remove columns that don't add much predictive power to the model. For example, columns that are uncorrelated with the target won't help a linear regression model. It will also enable you to remove columns that are derived from the target, or otherwise cause overfitting.

#### Instructions:

- Use the [DataFrame.corr()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) method to compute pairwise correlations between only the numeric columns. Assign the result to correlations.
- Display the average_rating column of correlations, which shows how much the other columns correlate with the average_rating column.
- Do any of the correlations surprise you? Write up your thoughts in a markdown cell.
- Remove any columns that seem to be derived from the average_rating. The bayes_average_rating is an example of this.
- Remove any columns that don't seem to correlate at all with the average_rating column.

In [ ]:
correlations = board_games[numeric_columns].corr()
correlations.average_rating

The other ratings columns (users_rated, bayes_average_rating) obviously correlate with average_rating.

In [ ]:
exclude = ['users_rated', 'average_rating', 'bayes_average_rating', 
           'average_weight', 'minplayers', 'maxplayers', 'playingtime',
          'minplaytime', 'maxplaytime']
features = list(set(numeric_columns) - set(exclude))

features

## 5: Creating A Model

Now that you're done exploring the data, you're ready to create a linear regression model and make predictions for newly created board games.

Ordinarily, you'd want to split the data into training and testing sets, train the algorithm on the training set, and test its performance on the test set. In this case, because we haven't covered training and testing sets yet, you'll evaluate the performance of the model on the training set.

You'll fit a linear regression model to board_games, using the columns you think should be predictors, and average_rating as the target. You'll then generate predictions using the same predictors you used in the fitting process.

#### Instructions:

- Initialize a [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model, and assign it to the variable `reg`.
- Use the [LinearRegression.fit()](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.fit) method to set the predictor columns you want the model to use and set the target column to `average_rating`.
- Use the [LinearRegression.predict()](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict) method to make predictions using the columns of board_games that you think should be used as predictors.
    - The predictors you pass into LinearRegression.predict() should be the same predictors you passed into [LinearRegression.fit](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.fit).
    - Assign the result to `predictions`.
- Calculate the error metric that you chose earlier.
- Write up what the error value tells you in a markdown cell.

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(board_games[features], board_games.average_rating)
predictions = lr.predict(board_games[features])

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(board_games.average_rating, predictions)
mse

Error rate 2.28 above the standard deviation 1.58 which indicates that model may not perform well. 

## 6: Next Steps

That's it for the guided steps. We recommend downloading this notebook and building on it. Here are some potential next steps:

- Split the data into training and testing sets, and calculate error on the testing set.
- Try algorithms other than linear regression.
- Calculate new predictors based off the existing columns, such as:
    - Player range (maxplayers - minplayers)
    - Playing time range (maxplaytime - minplaytime)
    - Average number of ratings (total_owners / users_rated)
- Scrape the latest data from [BoardGameGeek](http://www.boardgamegeek.com/) to acquire more data.